<a href="https://colab.research.google.com/github/omprakash0224/AI_agents/blob/main/collaboration_financial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multi-agent Collaboration for Financial Analysis**

In [ ]:
!pip install \
crewai \
crewai_tools \
langchain \
langchain-community \
langchain-google-genai \
google-generativeai \

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

Import libraries, APIs and LLM

In [3]:
from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI

In [16]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
os.environ["SERPER_API_KEY"] = userdata.get("SERPER_API_KEY")
os.environ["GEMINI_MODEL_NAME"] = "gemini-2.5-flash"

**crewAI Tools**

In [13]:
from crewai_tools import SerperDevTool
from langchain_google_genai import ChatGoogleGenerativeAI

search_tool = SerperDevTool()

**Creating Agents**

In [14]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Analyze recent news, corporate announcements, "
         "and macroeconomic developments related to {stock_selection} "
         "using news search results. Identify sentiment, risks, "
         "and potential market impact WITHOUT using price data.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [search_tool],
    llm=ChatGoogleGenerativeAI(model=os.environ["GEMINI_MODEL_NAME"])
)

In [ ]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [search_tool],
    llm=ChatGoogleGenerativeAI(model=os.environ["GEMINI_MODEL_NAME"])
)

In [ ]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [search_tool],
    llm=ChatGoogleGenerativeAI(model=os.environ["GEMINI_MODEL_NAME"])
)

In [ ]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [search_tool],
    llm=ChatGoogleGenerativeAI(model=os.environ["GEMINI_MODEL_NAME"])
)

**Creating Tasks**

In [ ]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Using ONLY news search results (no price or technical data), "
        "analyze recent developments related to {stock_selection}. "
        "Assess sentiment, headline risks, and event-driven impacts."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [ ]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)

In [ ]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)

In [ ]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

**Creating the Crew**

In [ ]:
from crewai import Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent,
            trading_strategy_agent,
            execution_agent,
            risk_management_agent],

    tasks=[data_analysis_task,
           strategy_development_task,
           execution_planning_task,
           risk_assessment_task],
    manager_llm=ChatGoogleGenerativeAI(model=os.environ["GEMINI_MODEL_NAME"],temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

**Running the Crew**

In [ ]:
# Example data for kicking off the process (Indian Market)
financial_trading_inputs = {
    'stock_selection': 'TATASTEEL.NS',   # NSE ticker
    'initial_capital': '100000',         # INR
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Intraday Trading',
    'news_impact_consideration': True
}

In [ ]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)